In [64]:
# TensorFlow経由でデバイス設定の確認が可能です
from tensorflow.python.client import device_lib
device_lib.list_local_devices()
 

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9860466108825911745, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 12399412945691708520
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 1926863294579192065
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14912199066
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 2051369525050039526
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [0]:
# データ分析・操作用ライブラリ
import numpy as np
import pandas as pd
from collections import Counter
import itertools

# NLP用ライブラリ
from sklearn.feature_extraction.text import TfidfVectorizer

In [48]:
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab


Reading package lists...
Building dependency tree...
Reading state information...
sudo is already the newest version (1.8.21p2-3ubuntu1).
libmecab-dev is already the newest version (0.996-5).
mecab is already the newest version (0.996-5).
mecab-ipadic-utf8 is already the newest version (2.7.0-20070801+main-1).
python-mecab is already the newest version (0.99.6-2).
curl is already the newest version (7.58.0-2ubuntu3.7).
file is already the newest version (1:5.32-2ubuntu0.2).
git is already the newest version (1:2.17.1-1ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [63]:
!cat /etc/mecabrc 

;
; Configuration file of MeCab
;
; $Id: mecabrc.in,v 1.3 2006/05/29 15:36:08 taku-ku Exp $;
;
dicdir = /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd

; userdic = /home/foo/bar/user.dic

; output-format-type = wakati
; input-buffer-size = 8192

; node-format = %m\n
; bos-format = %S\n
; eos-format = EOS\n



In [0]:
!sed -e "s!/var/lib/mecab/dic/debian!/Users/yutotakahashi/mecab-ipadic-neologd!g" /etc/mecabrc > /etc/mecabrc.new


In [59]:
!cat /etc/mecabrc.new

;
; Configuration file of MeCab
;
; $Id: mecabrc.in,v 1.3 2006/05/29 15:36:08 taku-ku Exp $;
;
dicdir = /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd

; userdic = /home/foo/bar/user.dic

; output-format-type = wakati
; input-buffer-size = 8192

; node-format = %m\n
; bos-format = %S\n
; eos-format = EOS\n



In [0]:
!cp /etc/mecabrc /etc/mecabrc.org
!cp /etc/mecabrc.new /etc/mecabrc

In [61]:
!cat /etc/mecabrc 

;
; Configuration file of MeCab
;
; $Id: mecabrc.in,v 1.3 2006/05/29 15:36:08 taku-ku Exp $;
;
dicdir = /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd

; userdic = /home/foo/bar/user.dic

; output-format-type = wakati
; input-buffer-size = 8192

; node-format = %m\n
; bos-format = %S\n
; eos-format = EOS\n



In [54]:
!apt-get -q -y install swig 

Reading package lists...
Building dependency tree...
Reading state information...
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [55]:
!pip install mecab-python3

In [66]:
import MeCab
mecab = MeCab.Tagger('-Ochasen')
res = mecab.parse('メカブって神ってる')
print(res)


RuntimeError: ignored

In [0]:

# CSVの読み込み
wikiData = pd.read_csv("train.csv", encoding="utf-8")

# 読み込みデータの表示
wikiData.head()



In [0]:
# 形態素解析用関数
def tokenize(text):
    tokens = []
    tagger = MeCab.Tagger( "-d /Users/yutotakahashi/mecab-ipadic-neologd" )
    node = tagger.parseToNode(text)
    while node:
        if node.feature.split(",")[0] == "名詞":
                replace_node = re.sub( re.compile( "[!-/:-@[-`{-~]" ), "", node.surface )
                if replace_node != "" and replace_node != " ":
                    tokens.append(replace_node)
        node = node.next
    return tokens

In [0]:
#辞書は下記のコマンドでアップデート
#cd mecab-ipadic-neologd
#./bin/install-mecab-ipadic-neologd -n

In [0]:
# Numpy配列へ変換
X = wikiData.text.values
y = wikiData.target.values

In [0]:
words = []
for text in X:
    words.append(tokenize(text))